In [47]:
import os, pickle, time, random, logging, json, gc, warnings
from datetime import datetime
from tqdm import tqdm

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import AdamW, BertConfig, BertTokenizer
from transformers import get_linear_schedule_with_warmup
from transformers import EncoderDecoderModel
from transformers import BertModel

from nltk.translate.bleu_score import sentence_bleu
import spacy

warnings.simplefilter("ignore")

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [23]:
from pathlib import Path

In [97]:
bert_model = 'bert-large-uncased'
model_path = Path('./models')
data_path = Path('../data')
bert_path = model_path / bert_model

In [52]:
if not (model_path / bert_model / 'config.json').is_file():
    BertModel.from_pretrained(bert_model).save_pretrained(model_path / bert_model)

In [103]:
class Attention(nn.Module):
    def __init__(self, n_h_enc, n_h_dec, n_h_attention):
        super().__init__()
        self.attn = nn.Linear(n_h_enc + n_h_dec, n_h_attention)
        self.v = nn.Parameter(torch.rand(n_h_attention), requires_grad=True)
        
    def forward(self, key, queries):
        batch_size, src_len = queries.shape
        
        key = key.unsqueeze(1).repeat(1, src_len, 1)
        
        energy = torch.tanh(self.attn(torch.cat([key, queries], dim=2)))
        
        v = self.v.repeat(batch_size, 1).unsqueeze(2)
        
        attn = torch.bmm(energy, v).squeeze(2)
        
        return F.softmax(attn, dim=1)

class Decoder(nn.Module):
    def __init__(self, out_dims, emb_dims, n_h_enc, n_h_dec, num_layers, dropout, attn):
        super().__init__()
        self.out_dims = out_dims
        self.emb_dims = emb_dims
        self.n_h_enc = n_h_enc
        self.n_h_dec = n_h_dec
        self.n_layers = num_layers
        self.attn = attn
        
        self.dropout = nn.Dropout(dropout)
        self.embedding = nn.Embedding(out_dims, emb_dims)
        
        self.nn = nn.GRU(emb_dims, n_h_dec, batch_first=True, num_layers=num_layers, dropout=dropout)
        
    def forward(self, src, queries, hidden):
        src = src.unsqueeze(1)
        embedded = self.embedding(src)
        embedded = self.dropout(embedded)
        
        out, hidden = self.rnn(embedded, hidden)
        out = out.squeeze()
        
        a = self.attn(out, queries)
        a = a.unsqueeze(1)
        weighted = torch.bmm(a, queries)
        
        weighted = weighted.squeeze(1)
        
        out = self.out(torch.cat([out, weighted], dim=1))
        
        return out, hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, encoder_trained):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.encoder_trained = encoder_trained
        
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        input_ids, token_ids, attn_mask = src
        
        if self.encoder_trained:
            bert_hs = self.encoder(input_ids, token_type_ids=token_ids, attention_mask=attn_mask)
        else:
            with torch.no_grad():
                bert_hs = self.encoder(input_ids, token_type_ids=token_ids, attention_mask=attn_mask)
        
        bert_encodings = bert_hs[0]
        
        batch_size, max_len = trg.shape
        
        trg_vocab_size = self.decoder.out_dims
        
        outputs = torch.zeros(batch_size, max_len, trg_vocab_size).to(device)
        
        out = trg[:, 0]
        
        hidden = torch.zeros(self.decoder.num_layers, out.shape[0], self.decoder.n_h_dec).to(device)
        
        for t in range(1, max_len):
            out, hidden = self.decoder(out, bert_encodings, hidden)
            outputs[:, t] = out
            teacher_force = random.random() < teacher_forcing_ratio
            out = (trg[:, t] if teacher_force else output.max(1)[1])
        
        return outputs

In [10]:
# redo below
def enable_reproducibility(seed=69):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

def no_grad(model):
    for p in model.parameters():
        p.requires_grad = False
    return model

def no_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def no_parameters(model):
    return sum(p.numel() for p in model.parameters())

def model_size(model):
    return sum(p.element_size() * p.nelement() for p in model.parameters())

def save_checkpoint(name, epoch, model, optimizer, valid_loss, train_loss):
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'valid_loss': valid_loss,
            'train_loss': train_loss,
            }, name)

def load_checkpoint(filename):
    checkpoint = torch.load(filename)
    return checkpoint['epoch'], checkpoint['model_state_dict'],\
           checkpoint['optimizer_state_dict'], checkpoint['valid_loss'], checkpoint['train_loss']

def epoch_time(start_time, end_time):
    elapsed_secs = end_time - start_time
    elapsed_mins = elapsed_secs / 60
    return elapsed_mins, elapsed_secs

In [11]:
pw_criterion = nn.CrossEntropyLoss(ignore_index=0)

def train(model, device, dataloader, optimizer, criterion, clip):
    log = logging.getLogger(__name__)
    model.train()

    epoch_loss = 0

    start = time.time()
    for i, (input_, output_) in enumerate(dataloader):

        input_data, input_length = input_
        output_data, output_length = output_

        optimizer.zero_grad()

        prediction = model([x.to(device) for x in input_data],  output_data.to(device))

        trg_sent_len = prediction.size(1)

        prediction = prediction[:, 1:].contiguous().view(-1, prediction.shape[-1])
        output_data = output_data[:, 1:].contiguous().view(-1)  # Find a way to avoid calling contiguous

        with torch.no_grad():
            pw_loss = pw_criterion(prediction,  output_data.to(device))

        loss = criterion(prediction,  output_data.to(device))

        # reshape to [trg sent len - 1, batch size]
        loss = loss.view(-1, trg_sent_len - 1)
        loss = loss.sum(1)
        loss = loss.mean(0)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.decoder.parameters(), clip)
        optimizer.step()

        if i % int(len(dataloader) * 0.1) == int(len(dataloader) * 0.1) - 1:
            log.info(
                f'Batch {i} Sentence loss {loss.item()} Word loss {pw_loss.item()}   Time: {epoch_time(start, time.time())}')
            start = time.time()

        epoch_loss += pw_loss.item()

    return epoch_loss / len(dataloader)

In [13]:
tokenizer = BertTokenizer.from_pretrained(bert_model)

def bleu_score(prediction, ground_truth):
    prediction = prediction.max(2)[1]
    acc_bleu = 0

    for x, y in zip(ground_truth, prediction):
        x = tokenizer.convert_ids_to_tokens(x.tolist())
        y = tokenizer.convert_ids_to_tokens(y.tolist())
        idx1 = x.index('[PAD]') if '[PAD]' in x else len(x)
        idx2 = y.index('[SEP]') if '[SEP]' in y else len(y)

        acc_bleu += bleu([x[1:idx1 - 1]], y[1:idx2 - 1], smoothing_function=SmoothingFunction().method4)
    return acc_bleu / prediction.size(0)

def eval(model, device, dataloader, criterion):
    log = logging.getLogger(__name__)
    model.eval()

    epoch_loss = 0
    epoch_bleu = 0

    with torch.no_grad():

        for i, (input_, output_) in enumerate(dataloader):

            input_data, input_length = input_
            output_data, output_length = output_

            prediction = model([x.to(device) for x in input_data], output_data.to(device), 0)  # turn off teacher forcing

            sample_t = tokenizer.convert_ids_to_tokens(output_data[0].tolist())
            sample_p = tokenizer.convert_ids_to_tokens(prediction[0].max(1)[1].tolist())
            idx1 = sample_t.index('[PAD]') if '[PAD]' in sample_t else len(sample_t)
            idx2 = sample_p.index('[SEP]') if '[SEP]' in sample_p else len(sample_p)

            bleu = bleu_score(prediction, output_data.to(device))

            trg_sent_len = prediction.size(1)
            # trg = [trg sent len, batch size]
            # output = [trg sent len, batch size, output dim]

            prediction = prediction[:, 1:].contiguous().view(-1, prediction.shape[-1])
            output_data = output_data[:, 1:].contiguous().view(-1)  # Find a way to avoid calling contiguous

            # trg = [(trg sent len - 1) * batch size]
            # output = [(trg sent len - 1) * batch size, output dim]

            pw_loss = pw_criterion(prediction, output_data.to(device))

            loss = criterion(prediction, output_data.to(device))
            loss = loss.view(-1, trg_sent_len - 1)
            loss = loss.sum(1)
            loss = loss.mean(0)

            if i % int(len(dataloader) * 0.1) == int(len(dataloader) * 0.1) - 1:
                log.info(f'Batch {i} Sentence loss: {loss.item()} Word loss: {pw_loss.item()} BLEU score: {bleu}\n'
                         f'Target {sample_t[1:idx1-1]}\n'
                         f'Prediction {sample_p[1:idx2-1]}\n\n')

            epoch_loss += pw_loss.item()
            epoch_bleu += bleu

        return epoch_loss / len(dataloader), epoch_bleu / len(dataloader)

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [54]:
LENGTH = 512

In [84]:
class BertDataset(Dataset):
    def __init__(self, json_path, bert_model):
        all_data = json.load(open(json_path, 'r'))
        input, output = self._extract_data(all_data)
        self.data = self._tokenize_data(input, output, bert_model)

    def _extract_data(self, all_data):
        input, output = [], []
        for data in all_data:
            input.append((data['passages'], data['responses']))
            output.append(data['clues'])
        return input, output

    def _tokenize_data(self, input, output, bert_model):
        tokenizer = BertTokenizer.from_pretrained(bert_model)
        
        data = tokenizer.batch_encode_plus(input, pad_to_max_length=True, return_tensors='pt')
        out_dict = tokenizer.batch_encode_plus(output, pad_to_max_length=True, return_tensors='pt')
        
        data['output_ids'] = out_dict['input_ids']
        data['output_len'] = out_dict['attention_mask'].sum(dim=1)
        data['input_len'] = data['attention_mask'].sum(dim=1)
        
        idx = (data['input_len'] <= LENGTH)
        in_m = max(data['input_len'][idx])
        out_m = max(data['output_len'][idx])
        
        data['input_ids'] = data['input_ids'][idx, :in_m]
        data['attention_mask'] = data['attention_mask'][idx, :in_m]
        data['token_type_ids'] = data['token_type_ids'][idx, :in_m]
        data['input_len'] = data['input_len'][idx]
        
        data['output_ids'] = data['output_ids'][idx, :out_m]
        data['output_len'] = data['output_len'][idx]
        
        return data
    
    def __len__(self):
        return self.data['input_ids'].shape[0]
    
    def __getitem__(self, idx):
        return (((self.data['input_ids'][idx],
                self.data['attention_mask'][idx],
                self.data['token_type_ids'][idx]),
                self.data['input_len'][idx]),
                (self.data['output_ids'][idx],
                self.data['output_len'][idx])
                )

In [86]:
train_set = BertDataset('data_train.json', bert_model)
dev_set = BertDataset('data_dev.json', bert_model)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1241 > 512). Running this sequence through the model will result in indexing errors


In [99]:
class HP:
    def __init__(self):
        self.num_workers = 0
        self.pin_memory = False
        self.batch_size = 4
        self.weight_decay = 0.001
        self.lr = 0.05
        self.decoder_hidden_size = 512
        self.decoder_input_size = 512
        self.attention_hidden_size = 512
        self.n_layers = 1
        self.clip = 1
        self.dropout = 1
        self.epochs = 4
        self.mb = 32
        self.checkpoint = None
        self.encoder_trained = False
        
        conf_file = json.load(open((bert_path / 'config.json'), 'r'))
        self.bert_hidden_size = conf_file['hidden_size']
        self.bert_vocab_size = conf_file['vocab_size']
        
        
hp = HP()

In [100]:
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=hp.num_workers, pin_memory=hp.pin_memory
                         )
dev_loader = DataLoader(dev_set, batch_size=hp.batch_size, shuffle=True,
                        num_workers=hp.num_workers, pin_memory=hp.pin_memory
                       )

In [104]:
attn = Attention(hp.bert_hidden_size, hp.decoder_hidden_size, hp.attention_hidden_size)
decoder = Decoder(hp.bert_vocab_size, hp.decoder_input_size, hp.bert_hidden_size,
                  hp.decoder_hidden_size, hp.n_layers, hp.dropout, attn)
encoder = BertModel.from_pretrained(bert_path)

In [ ]:
model = Seq2Seq()

In [85]:
ds = BertDataset('../../data/bert/toy')
#train_set, batch_size=mb, shuffle=True, num_workers=dl_workers, pin_memory=True if device=='cuda' else False
dl = DataLoader(ds, batch_size=8, num_workers=1, shuffle=True, pin_memory=True)
src, dest = next(iter(dl))
src, src_len = src

print(src[0][0].shape)
print(src[1][0].shape)
print(src[2][0].shape)

TypeError: __init__() missing 1 required positional argument: 'bert_model'

In [15]:
train_set = ...
dev_set = ...

In [16]:
train_loader = ...
dev_loader = ...

In [ ]:
...

In [17]:
attention = Attention(bert_hidden_size, decoder_hidden_size, attn_hidden_size)
decoder = Decoder(bert_vocab_size, decoder_input_size, bert_hidden_size, decoder_hidden_size, num_layers,
                 dropout, attention)

NameError: name 'bert_hidden_size' is not defined

In [ ]:
encoder = BertModel.from_pretrained('...')

In [ ]:
model = Seq2Seq(encoder, decoder, encoder_trained)

In [22]:
optimizer = optim.SGD(decoder.paramters(), weight_decay=weight_decay, lr=lr, momentum=momentum)
criterion = nn.CrossEntropyLoss(ignore_index=0, reduction='none')

NameError: name 'decoder' is not defined

In [ ]:
if checkpoint is not None:
    last_epoch, model_dict, optim_dict, valid_loss_list, train_loss_list = load_checkpoint(checkpoint)
    last_epoch += 1
    model.load_state_dict(model_dict)
    
    best_valid_loss = min(valid_loss_list)
    
    optimizer.load_state_dict(optim_dict)
    
    for state in optimizer.state.values():
        for k, v in state.items():